In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
df = pd.read_excel("data/santafe_2008-2011.xlsx")
print(len(df))
print(df.media.value_counts())

161
media
rosario12      75
lacapital      35
ellitoral      30
elciudadano    21
Name: count, dtype: int64


In [34]:
def content_extract(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        # Decode the content as utf-8 explicitly
        html_content = response.content.decode('utf-8', errors='ignore')
        soup = BeautifulSoup(html_content, 'html.parser')
        content = soup.findAll('p')
        content_list = [p.get_text(strip=True) for p in content]
        
        # Apply rules based on text in the URL
        if 'lacapital.com' in url:
            content_list = content_list[1:-1]  # Discard the first and last
        elif 'pagina12.com' in url:
            content_list = content_list[3:-2]  # Discard the first 3 and last 2
        elif 'elciudadanoweb.com' in url:
            content_list = content_list[:-2]  # Discard the last 2
        
        # Return the processed list or a default message
        return content_list if content_list else ["No content found"]
    except requests.exceptions.RequestException as e:
        return [f"Request error: {e}"]
    except Exception as e:
        return [f"Error: {e}"]

# Apply the function to the 'links' column and create a new column 'content'
df['content'] = df['link'].apply(content_extract)


In [37]:
df.to_excel("data/test.xlsx")

In [38]:
df_short = df.copy()

# Function to filter out short strings from the lists
def filter_short_strings(content_list):
    return [text for text in content_list if len(text) >= 100]

# Apply the function to the "content" column
df_short['content'] = df_short['content'].apply(filter_short_strings)


In [44]:
df_short['date'] = df_short['date'].astype(str)

In [50]:
df_short.drop([108, 141, 142], inplace=True)

In [51]:
df_short.to_json("data/scraped_content.json", orient="records", lines=True)